# Preamble

In [1]:
HomeDir = '../'
DataDir = HomeDir+'data/'
ListDir = HomeDir+'lists/'
ListResDir = HomeDir+'lists/sim/'

import astropy.units as u
from astropy import constants as const
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle
import healpy as hp
import numpy as np
import math
import copy
from tqdm import tqdm, tqdm_notebook
from time import time as tictoc
import pandas as pd
import scipy
from scipy.interpolate import griddata
from scipy import interpolate
from scipy import integrate
import os
import sys
import importlib
from scipy.optimize import fsolve
import scipy.stats as st
from scipy.spatial.transform import Rotation as R

from IPython.core.display import display, HTML
from IPython.display import display, clear_output
display(HTML("<style>.container { width:100% !important; }</style>"))
np.set_printoptions(edgeitems=3, linewidth=200) 
pd.set_option('display.max_columns', None)
pd.set_option('max_rows',200) and pandas.set_option('max_columns',20)

from MyUnits import *

# Functions definition

## Class for the sky patch

In [2]:
class sky_patch:
    """Class defining the stellar target properties. Notice that this works only for a disk on the sky."""
    def __init__(self, center_ra, center_dec, disc_radius, distance, data_file_name, mu_bcrs=np.array([0,0])):
        self.center_ra = center_ra
        self.center_dec = center_dec
        self.disc_radius = disc_radius
        self.distance = distance # distance from the observer
        ### galactic coordinates of the center (in deg)
        self.center_l = SkyCoord(center_ra*u.deg, center_dec*u.deg, frame = 'icrs').galactic.l.radian/degree 
        self.center_b = SkyCoord(center_ra*u.deg, center_dec*u.deg, frame = 'icrs').galactic.b.radian/degree 
        ### solid angle covered by the stellar target (in radians)
        self.delta_omega = 2*np.pi*(1-math.cos(disc_radius)) #np.pi*disc_radius**2   
        ### proper motion of the stellar target in the Barycentric Celestial Reference Systems (aligned with ICRS)
        self.mu_bcrs = mu_bcrs # in mas/y
        self.data_file_name = data_file_name

## Angular separation

In [3]:
def angular_sep_magn_sq(ra1, dec1, ra2, dec2):
    """Computes the magnitude of the angular separation vector"""
    return np.arccos(np.sin(dec1)*np.sin(dec2) + np.cos(dec1)*np.cos(dec2)*np.cos(ra2-ra1), out=np.zeros(len(ra2)), where=((ra1!=ra2) & (dec1!=dec2)) )**2

def angular_sep(ra1, dec1, ra2, dec2):
    """Computes 2d angular separations vector for stars close to each other"""
    return np.array([(ra1-ra2)*np.cos((dec1+dec2)/2), (dec1-dec2)]).T

def angular_sep_scalar(ra1, dec1, ra2, dec2):
    """Computes the magnitude of the angular separation vector for stars close to each other"""
    return np.sqrt( ((ra1-ra2)*np.cos((dec1+dec2)/2))**2 + (dec1-dec2)**2 )

## Equatorial to ecliptic coordinate transformation

From https://gea.esac.esa.int/archive/documentation/GEDR3/Gaia_archive/chap_datamodel/sec_dm_main_tables/ssec_dm_gaia_source.html and section 1.5.3 of https://www.cosmos.esa.int/documents/532822/552851/vol1_all.pdf

In [4]:
rot_matrix = np.array([[1, 0, 0], [0, 0.9174821334228558, 0.39777699135300065], [0, -0.39777699135300065, 0.9174821334228558]])
ra_offset = 0.05542*arcsec
  
def fn_eq_to_ecl_array(ra, dec):
    """Function to convert the equatorial coordinates (ra, dec) to ecliptic longitude and latitude according to the Gaia reference frame"""
    """Takes angle in deg and returns in deg"""    
    ra_s, dec_s = ra*degree + ra_offset, dec*degree
    x_vec_eq = np.array([np.cos(dec_s)*np.cos(ra_s), np.cos(dec_s)*np.sin(ra_s), np.sin(dec_s)])
    x_vec_ecl = (rot_matrix @ x_vec_eq).T
    
    if np.isscalar(ra):
        ecl_lon, ecl_lat = (np.arctan2(x_vec_ecl[1], x_vec_ecl[0])), np.arctan2(x_vec_ecl[2], np.sqrt(x_vec_ecl[0]**2 + x_vec_ecl[1]**2))
        ecl_lon = ecl_lon + 2*np.pi*np.heaviside(-ecl_lon, 0) ### shift the interval from [-pi, pi] to [0, 2*pi]        
    else:
        ecl_lon, ecl_lat = (np.arctan2(x_vec_ecl[:, 1], x_vec_ecl[:, 0])), np.arctan2(x_vec_ecl[:, 2], np.sqrt(x_vec_ecl[:, 0]**2 + x_vec_ecl[:, 1]**2))
        ecl_lon = ecl_lon + 2*np.pi*np.heaviside(-ecl_lon, 0) ### shift the interval from [-pi, pi] to [0, 2*pi]

    return ecl_lon/degree, ecl_lat/degree

## Healpy map and list of neighbors for each pixel

In [5]:
def fn_nb_pixel(patch_pix, radius_nb, nside, nest=True):
    """For healpy pixels in the 1d array patch_pix returns the neighbors of each pixel (disc of radius radius_nb in rad)"""    
    ### Cartesian vectors for each pixel in patch_pix
    vec_pix_x, vec_pix_y, vec_pix_z = hp.pix2vec(nside, patch_pix, nest=nest) 
    vec_array = np.array([vec_pix_x, vec_pix_y, vec_pix_z]).T
    
    ### Loop over pixels
    nb_pix = []; 
    for i in range(len(patch_pix)):
        ### Disc around the pixel position  
        nb_pix.append(hp.query_disc(nside, vec_array[i], radius_nb, inclusive=True, nest=nest))      
    return nb_pix

### Notice that search_around_sky is not faster than this function if we need to query neighbors within 0.1-0.3 degree. Healpy pixelation is more efficient in that case. 

## Beta_t optimal

In [6]:
rs_MW = 18*kpc ### r_s parameter of the NFW profile of the Milky Way (scale radius)
rho_s_MW = 0.003*MSolar/pc**3 ### rho_s  parameter of the NFW profile of the Milky Way (scale density)
d_Sun = 8.29*kpc ### distance of the Sun from the Galactic Center

def fn_rho_dm(dist, l, b): ### works with scalar quantities
    """Dark matter energy density at distance dist and in the direction (l, b), given in Galactic coordinates"""
    r_vec_sun = np.array([0, d_Sun, 0]) ### 3d vector position of the Sun wrt the Galactic Center    
    r_vec = dist*np.array([np.sin(l)*np.cos(b), np.cos(l)*np.cos(b), np.sin(b)]) - r_vec_sun ### 3d vector wrt the Galactic Center
    r_over_rs = np.linalg.norm(r_vec/rs_MW)
    
    return 4*rho_s_MW/(r_over_rs*(1+r_over_rs)**2)

def fn_rho_dm_array(dist, l, b): ### works with numpy arrays 
    """Dark matter energy density at distance dist and in the direction (l, b), given in Galactic coordinates"""
    r_vec_sun = np.full((len(dist), 3), np.array([0, d_Sun, 0])) ### 3d vector position of the Sun wrt the Galactic Center  
    r_vec = np.array([dist*np.sin(l)*np.cos(b) - r_vec_sun[:, 0],
                      dist*np.cos(l)*np.cos(b) - r_vec_sun[:, 1],
                      dist*np.sin(b) - r_vec_sun[:, 2]]).T  ### 3d vector wrt the Galactic Center
    r_over_rs = np.linalg.norm(r_vec/rs_MW, axis=1)
    
    return 4*rho_s_MW/(r_over_rs*(1+r_over_rs)**2)

In [7]:
def fn_n_lens(M_l, f_l, dist, l, b, delta_omega):
    """Average number of lenses with mass M_l and fractional abundance f_l in front of a stellar target centered at Galactic Coordinates (l, b) and covering a solid angle delta_omega, up to a distance dist"""
    integrand = lambda dist: dist**2*fn_rho_dm(dist, l, b) ### function to be integrated over distance        
    return delta_omega*f_l*integrate.quad(integrand, 0, dist)[0]/M_l

def fn_n_lens_tot(M_l, f_l, dist, sky_patches, dist_max=False):
    """Total number of lenses in front of all the sky patches up to a distance dist, if dist_max=False"""
    """If dist_max=True, the distance is set to the distance of the stellar target for each patch"""
    n_lens_tot = 0
        
    for i in range(len(sky_patches)):
        if dist_max==False:
            n_lens_tot += fn_n_lens(M_l, f_l, dist, sky_patches[i].center_l*degree, sky_patches[i].center_b*degree, sky_patches[i].delta_omega)    
        else:
            n_lens_tot += fn_n_lens(M_l, f_l, sky_patches[i].distance, sky_patches[i].center_l*degree, sky_patches[i].center_b*degree, sky_patches[i].delta_omega)    
            
    return n_lens_tot

In [8]:
def fn_beta_t_opt(M_l, r_l, f_l, sky_patches):
    """Find the beta_t optimal for the given lens population parameters and the given stellar targets"""
    n_lens_opt = 3 ### number of optimal lenses with beta_t >= beta_t_optimal
    n_lens_max = fn_n_lens_tot(M_l, f_l, 0, sky_patches, dist_max=True) ### total number of lenses in front of the stellar targets
        
    if n_lens_max>=n_lens_opt:
        d_min_list = np.zeros((len(sky_patches)))        
        for i in range(len(sky_patches)):
            d_min_list[i] = (3*M_l/(sky_patches[i].delta_omega*rho_s_MW))**(1/3)
        dist_solve = lambda d : (fn_n_lens_tot(M_l, f_l, d, sky_patches, dist_max=False) - n_lens_opt)
        d_opt = fsolve(dist_solve, np.min(d_min_list))[0] # solve for dist_solve==0, starting from the minimum distance at which there is one lens
    else:
        ### take the average of the distance to each patch on the sky
        d_opt = 0
        for i in range(len(sky_patches)):
            d_opt += sky_patches[i].distance
        d_opt = d_opt/len(sky_patches)
        
    return r_l/d_opt ### optimal beta_t in radians

## Lens population

In [9]:
v0_ss = 238*1000*Meter/Second ### Solar System velocity, i.e. the observer velocity wrt the DM halo
sigma_vl = 166*1000*Meter/Second #v0_ss/math.sqrt(2) ### Dark Matter velocity dispersion

def fn_3d_unit_vec(th, phi):
    """Unit vector in 3d"""
    return np.array([np.sin(th)*np.cos(phi), np.sin(th)*np.sin(phi), np.cos(th)]).T

def fn_lens_population(M_l, f_l, sky_patch):
    """Generate a random population of lenses in front of the stellar target"""
    ### Random number of lenses in front of the stellar target with Poisson distribution
    n_lens_avg = fn_n_lens(M_l, f_l, sky_patch.distance, sky_patch.center_l*degree, sky_patch.center_b*degree, sky_patch.delta_omega)    
    n_lens = np.random.poisson(n_lens_avg)
    
    ### Probability distribution function for the polar angle theta within the region covered by the stellar target
    class pdf_theta(st.rv_continuous):
        def _pdf(self, th):
            return np.sin(th)/(1-np.cos(sky_patch.disc_radius))
    theta_dist = pdf_theta(a=0, b=sky_patch.disc_radius, name='pdf_theta')

    ### Random lens position in front of the stellar target
    theta_lens = theta_dist.rvs(size=n_lens)
    phi_lens = np.random.uniform(0, 2*np.pi, n_lens)
    ### Rotation in the direction of the stellar target
    vec_center_patch = fn_3d_unit_vec(np.pi/2-sky_patch.center_dec*degree, sky_patch.center_ra*degree)
    rot_unit_vector = np.cross(np.array([0, 0, 1]), vec_center_patch) ### direction of the rotation axis, must be normalized
    rot_unit_vector = rot_unit_vector/np.sqrt(rot_unit_vector[0]**2 + rot_unit_vector[1]**2 + rot_unit_vector[2]**2)
    rot_matrix = R.from_rotvec(np.arccos(np.dot(np.array([0, 0, 1]), vec_center_patch)) * rot_unit_vector)

    rot_lens_vectors = rot_matrix.apply(fn_3d_unit_vec(theta_lens, phi_lens))
    lens_ra = (np.arctan2(rot_lens_vectors[:, 1], rot_lens_vectors[:, 0]))/degree
    lens_dec = (np.pi/2-np.arctan2(np.sqrt(rot_lens_vectors[:, 0]**2 + rot_lens_vectors[:, 1]**2), rot_lens_vectors[:, 2]))/degree 
        
    ### Random lens velocities
    vl_ra = np.random.normal(0, sigma_vl, n_lens)
    vl_dec = np.random.normal(0, sigma_vl, n_lens)
        
    ### Probability distribution function for the lens distance, assuming the DM energy density along the line of sight towards the center of the stellar target    
    class pdf_dist(st.rv_continuous):
        def _pdf(self, d):
            return 1/n_lens_avg*sky_patch.delta_omega*f_l*d**2*fn_rho_dm(d, sky_patch.center_l*degree, sky_patch.center_b*degree)/M_l 
    d_dist = pdf_dist(a=0, b=sky_patch.distance, name='pdf_dist')
    
    dl = d_dist.rvs(size=n_lens)/kpc
    
    return n_lens, np.array([lens_ra, lens_dec, vl_ra, vl_dec, dl]).T  

## Noise injection

Given a list of stars, compute the pm and parallax noise and update the stars pm and parallax.

Strategy:
- Start from the stars after the first cleaning, but before the iterative background field subtraction. Use the columns (pmra_sub, pmdec_sub, parallax_sub).
- Inject the noise by computing the noise PDFs in different g magnitude and radial bins and extract random values for (pm_ra, pm_dec, parallax).
- Save the mock proper motion and parallax in the new columns (pmra_sim, pmdec_sim, parallax_sim).

In [10]:
def fn_noise_inj(data, disc_center, gmag_bin_size=0.1, rad_bin_size=1, noise=True):
    """Data-driven injectiong of the proper motion and parallax noise"""
    ### Simulated pm_ra, pm_dec, and parallax
    pmra_sim, pmdec_sim, parallax_sim = np.zeros(len(data)), np.zeros(len(data)), np.zeros(len(data))    

    if noise:
        print('Injecting the data-driven noise..')
        ### Bin in g magnitude 
        data_g = data['phot_g_mean_mag'].to_numpy()
        min_g, max_g = np.min(data_g), np.max(data_g)
        ### Using g magnitude bins with approximately equal number of stars per bin, to avoid bins with a low number of stars    
        n_bins_g = int(np.ceil((max_g-min_g)/gmag_bin_size))
        bins_g = np.interp(np.linspace(0, len(data_g+1), n_bins_g + 1), np.arange(len(data_g)+1), np.append(np.sort(data_g), max_g*1.001))  # make sure that the last bin includes max_g
        q_bin_g = np.digitize(data_g, bins_g)-1         

        ### Bin in radial distance from the center        
        center_sky_coord = SkyCoord(ra = disc_center[0] * u.deg, dec = disc_center[1] * u.deg)
        data_sky_coord = SkyCoord(ra = data['ra'].to_numpy() * u.deg, dec = data['dec'].to_numpy() * u.deg)
        data_r = data_sky_coord.separation(center_sky_coord).value
        bins_r = np.arange(0, np.max(data_r)+rad_bin_size, rad_bin_size)
        q_bin_r = np.digitize(data_r, bins_r)-1     
        
        ### Group the stars according to their g mag and radial position
        df_groupby = pd.DataFrame({'q_bin_g':q_bin_g, 'q_bin_r':q_bin_r,
                                   'pmra_sub':data['pmra_sub'].to_numpy(), 'pmdec_sub':data['pmdec_sub'].to_numpy(),
                                   'parallax_sub':data['parallax_sub'].to_numpy()}).groupby(by=['q_bin_g', 'q_bin_r'], as_index=False)        
         
        n_bins = 80 ### number of bins to build the PDFs

        for r in np.arange(np.min(q_bin_r), np.max(q_bin_r)+1):
            for g in np.arange(np.min(q_bin_g), np.max(q_bin_g)+1):
                data_group = df_groupby.get_group((g, r))
                group_index = np.array(list(data_group.index)) # indices for the stars in this group
                n_stars = len(data_group)

                pdf_pmra, pmra_edges = np.histogram(data_group['pmra_sub'].to_numpy(), bins=n_bins, density=True)
                pdf_pmdec, pmdec_edges = np.histogram(data_group['pmdec_sub'].to_numpy(), bins=n_bins, density=True)
                pdf_parallax, parallax_edges = np.histogram(data_group['parallax_sub'].to_numpy(), bins=n_bins, density=True)

                bin_step_pmra = pmra_edges[1:] - pmra_edges[:-1]
                bin_step_pmdec = pmdec_edges[1:] - pmdec_edges[:-1]
                bin_step_par = parallax_edges[1:] - parallax_edges[:-1]

                ### Get the CDFs and interpolate the inverse CDFs
                cdf_pmra = np.cumsum(pdf_pmra)*bin_step_pmra
                inv_cdf_pmra = scipy.interpolate.interp1d(cdf_pmra, pmra_edges[1:])
                pmra_sim[group_index] = inv_cdf_pmra(np.random.uniform(cdf_pmra[0], cdf_pmra[-1], n_stars))

                cdf_pmdec = np.cumsum(pdf_pmdec)*bin_step_pmdec
                inv_cdf_pmdec = scipy.interpolate.interp1d(cdf_pmdec, pmdec_edges[1:])
                pmdec_sim[group_index] = inv_cdf_pmdec(np.random.uniform(cdf_pmdec[0], cdf_pmdec[-1], n_stars))

                cdf_par = np.cumsum(pdf_parallax)*bin_step_par
                inv_cdf_par = scipy.interpolate.interp1d(cdf_par, parallax_edges[1:])
                parallax_sim[group_index] = inv_cdf_par(np.random.uniform(cdf_par[0], cdf_par[-1], n_stars))            
    else:
        print('Skipping noise injection. Setting stars proper motion and parallax to zero.')

    ### Add columns for the simulated pm and parallax
    data.insert(len(data.columns), 'pmra_sim', pmra_sim)
    data.insert(len(data.columns), 'pmdec_sim', pmdec_sim)
    data.insert(len(data.columns), 'parallax_sim', parallax_sim)

    return None

## Signal injection

### Observer and star velocity

In [11]:
### The velocity of the observer in given by the velocity of the Sun in a reference frame where the galaxy is at rest:
### v_sun = 238 Km/s in the direction l = 270 deg, b = 0 (in Galactic coordinates)
### or alpha = 138 deg, dec = -48.33 deg (in Equatorial coordinates)
### Check i.e. with:
### v_sun_dir = SkyCoord(v_sun_ra*u.rad, v_sun_dec*u.rad)
### v_sun_dir.galactic

v_sun = 238*math.pow(10,3)*Meter/Second ### magnitude of the Sun velocity
v_sun_ra, v_sun_dec = 138.00438151*degree, -48.32963721*degree ### direction of the Sun velocity in equatorial coordnates
v_obs = v_sun*fn_3d_unit_vec(np.pi/2-v_sun_dec, v_sun_ra)

def fn_obs_vel(ra, dec):
    """Returns the observer velocity perpendicular to the line of sight towards the direction (ra, dec)"""
    theta, phi = np.pi/2 - dec, ra
    u_th = np.array([np.cos(theta)*np.cos(phi), np.cos(theta)*np.sin(phi), -np.sin(theta)]).T
    if np.isscalar(theta):
        u_phi = np.array([-np.sin(phi), np.cos(phi), 0]).T
    else:
        u_phi = np.array([-np.sin(phi), np.cos(phi), np.zeros(len(theta))]).T

    return np.array([np.inner(v_obs, u_phi), -np.inner(v_obs, u_th)]).T

### The velocity of the star in a reference frame where the galaxy is at rest is given by 
### the proper motion of the stellar target in the Barycentric Celestial Reference Systems (aligned with ICRS)
### multiplied by the stellar target distance and added to the observer velocity with rispect to the Galactic center,
### as computed by the function fn_obs_vel

def fn_star_vel(v_obs, mu_star_bcrs, distance):
    """Returns the star velocity with respect to the galactic center"""
    return mu_star_bcrs*distance + v_obs

### Matched filters

In [12]:
# Uploading lists for the G_0 function and it's derivative. G_0 is the enclosed lens mass within a cylinder oriented along the line of sight. 
# See Eq.(3.10)=(3.11) of 1804.01991 or Eq.(2) of 2002.01938
# For the NFW truncated lens profile given by Eq.(3) of 2002.01938 the enclosed mass cannot be computed analytically. We use an interpolation function.

logxG0_list = np.loadtxt(ListDir+'G0NFWtrunc.txt');  logxG0_prime_list = np.loadtxt(ListDir+'G0pNFWtrunc.txt');  #logxG0_second_list = np.loadtxt(ListDir+'G0ppNFWtrunc.txt');
logG0_fnc = interpolate.interp1d(logxG0_list[:, 0], logxG0_list[:, 1], kind='cubic', bounds_error=False, fill_value=(logxG0_list[0, 1], logxG0_list[-1, 1]))
logG0_p_fnc = interpolate.interp1d(logxG0_prime_list[:, 0], logxG0_prime_list[:, 1], kind='cubic', bounds_error=False, fill_value=(logxG0_prime_list[0, 1], logxG0_prime_list[-1, 1]))

"Returns the lens enclosed mass within the distance x = beta/beta_l"
def G0_fnc(x): return np.power(10, logG0_fnc(np.log10(x+1E-20)))
def G0_p_fnc(x): return np.power(10, logG0_p_fnc(np.log10(x+1E-20)))

def dipole_mf(b_l, b_vec):
    """Returns the pm dipole-like profile from Eq.(2) of 2002.01938"""
    b_norm = np.sqrt(b_vec[:, 0]**2 + b_vec[:, 1]**2)
    b_hat = np.array([b_vec[:,0]/(b_norm+1E-20), b_vec[:,1]/(b_norm+1E-20)]).T; x = b_norm/b_l
    G0_over_xsq = G0_fnc(x)/(x**2+1E-20); G0p_over_x = G0_p_fnc(x)/(x+1E-20)

    remove_inf = np.heaviside(b_norm, 0) # set to zero values corresponding to b_vec = [0, 0], remove infinity at the origin
    
    dipole_ra = np.array([(G0_over_xsq*(2*b_hat[:, 0]*b_hat[:, 0] - 1) - G0p_over_x*b_hat[:,0]*b_hat[:,0])*remove_inf, 
                          (G0_over_xsq*(2*b_hat[:, 1]*b_hat[:, 0]) - G0p_over_x*b_hat[:, 1]*b_hat[:, 0])*remove_inf]).T
    dipole_dec = np.array([(G0_over_xsq*(2*b_hat[:, 0]*b_hat[:, 1]) - G0p_over_x*b_hat[:, 1]*b_hat[:, 0])*remove_inf, 
                           (G0_over_xsq*(2*b_hat[:, 1]*b_hat[:, 1] - 1) - G0p_over_x*b_hat[:, 1]*b_hat[:, 1])*remove_inf]).T
    isotropic_dipole_magn = (G0_over_xsq**2 + 0.5*(G0p_over_x**2-2*G0_over_xsq*G0p_over_x))*remove_inf # for the normalization; see Eq. (13) of 2002.01938
            
    return dipole_ra, dipole_dec, isotropic_dipole_magn

In [13]:
def parallax_mf(b_l, b_vec, sinb):
    """Returns the parallax profile"""
    b_norm = np.sqrt(b_vec[:, 0]**2 + b_vec[:, 1]**2)
    b_hat = np.array([b_vec[:,0]/(b_norm+1E-20), b_vec[:,1]/(b_norm+1E-20)]).T; x = b_norm/b_l
    G0_over_xsq = G0_fnc(x)/(x**2+1E-20); G0p_over_x = G0_p_fnc(x)/(x+1E-20)
       
    return -(G0_over_xsq*(2*b_hat[:, 1]**2-1)*(1-sinb**2) + G0p_over_x*(1 - (1-sinb**2)*b_hat[:, 1]**2))/(1+sinb**2)   

### Signal injection

Goal: given a list of lenses and a list of stars in a specific field of view, compute the lens-induced proper motion and parallax and add it to the stars' pm.

Strategy:
- start from the simulated stars after noise injection
- fill in a sparse array of zero pm for each pixel and add the lens correction for each lens using the template mask
- update the list of stars adding the lens-corrected proper motion to the noise, based on their location on the grid

In [14]:
def fn_signal_inj(data, M_l, r_l, n_lens, lens_pop, sim_sky_patch, n_betat, min_beta_t=0.002*degree):
    """Returns the data with the signal given by the lenses in lens_pop added to the columns pmra_sim, pmdec_sim, and parallax_sim"""
    
    data_ra, data_dec = data['ra'].to_numpy(), data['dec'].to_numpy()
    data_ecl_lon, data_ecl_lat = data['ecl_lon'].to_numpy(), data['ecl_lat'].to_numpy()
    data_pmra_sim, data_pmdec_sim, data_parallax_sim = np.zeros((len(data))), np.zeros((len(data))), np.zeros((len(data))) 
    
    if n_lens==1:
        vec_lens_array = hp.ang2vec(lens_pop[0], lens_pop[1], lonlat=True) # vector for the location of the lense

        v_obs_lens = fn_obs_vel(lens_pop[0]*degree, lens_pop[1]*degree)
        v_star_lens = fn_star_vel(v_obs_lens, sim_sky_patch.mu_bcrs*mas/Year, sim_sky_patch.distance)
        Dl_over_Di = lens_pop[4]*kpc/sim_sky_patch.distance
        vil_ra = lens_pop[2] - (1-Dl_over_Di)*v_obs_lens[0] - Dl_over_Di*v_star_lens[0]
        vil_dec = lens_pop[3] - (1-Dl_over_Di)*v_obs_lens[1] - Dl_over_Di*v_star_lens[1]
        beta_l = r_l/(lens_pop[4]*kpc)

        lens_pop_ecl_lon, lens_pop_ecl_lat = fn_eq_to_ecl_array(lens_pop[0], lens_pop[1]) # convert into ecliptic coordinates for the parallax template

        ### To find the stars around the lens use a pixelation scale of size approx. beta_l/10 and keep stars withing n_betat*beta_l
        max_beta_l = max(beta_l, min_beta_t)
        n = round(math.log(np.sqrt(np.pi/3)/(0.1*max_beta_l), 2)); nside = 2**n; 
        q_pix = np.asarray(hp.ang2pix(nside, data_ra, data_dec, nest=True, lonlat=True)) 

        ### Find stars around the lens
        nb_lens_i = hp.query_disc(nside, vec_lens_array, n_betat*max_beta_l, inclusive=True, nest=True)
        stars_in = ((q_pix >= nb_lens_i[0]) & (q_pix <= nb_lens_i[-1])) # first reduce the total number of stars
        nb_stars = np.isin(q_pix[stars_in], nb_lens_i, assume_unique=False, invert=False) # keep only stars within the neighboring pixels  

        ### Proper motion template
        beta_it = angular_sep(lens_pop[0]*degree, lens_pop[1]*degree, data_ra[stars_in][nb_stars]*degree, data_dec[stars_in][nb_stars]*degree)
        mura_tilde, mudec_tilde, mu_sq = dipole_mf(beta_l, beta_it)
        mu_signal = (1-Dl_over_Di)*4*GN*M_l*vil_ra/r_l**2*mura_tilde/(mas/Year) + (1-Dl_over_Di)*4*GN*M_l*vil_dec/r_l**2*mudec_tilde/(mas/Year) # in mas/y

        data_pmra_sim[np.where(stars_in)[0][nb_stars]] += mu_signal[:, 0] # this way of accessing the elements of data_pmra_sim does not make a copy of the array
        data_pmdec_sim[np.where(stars_in)[0][nb_stars]] += mu_signal[:, 1]

        ### Parallax template
        beta_it_ecl = angular_sep(lens_pop_ecl_lon*degree, lens_pop_ecl_lat*degree, data_ecl_lon[stars_in][nb_stars]*degree, data_ecl_lat[stars_in][nb_stars]*degree)
        par_t = parallax_mf(beta_l, beta_it_ecl, np.sin(lens_pop_ecl_lat*degree)) 

        data_parallax_sim[np.where(stars_in)[0][nb_stars]] += (1-Dl_over_Di)*4*GN*M_l*AU/r_l**2*par_t/mas # in mas
        
    else:    
        vec_lens_array = hp.ang2vec(lens_pop[:, 0], lens_pop[:, 1], lonlat=True) # vector for the location of the lenses

        v_obs_lens = fn_obs_vel(lens_pop[:, 0]*degree, lens_pop[:, 1]*degree)
        v_star_lens = fn_star_vel(v_obs_lens, sim_sky_patch.mu_bcrs*mas/Year, sim_sky_patch.distance)
        Dl_over_Di = lens_pop[:, 4]*kpc/sim_sky_patch.distance
        vil_ra = lens_pop[:, 2] - (1-Dl_over_Di)*v_obs_lens[:, 0] - Dl_over_Di*v_star_lens[:, 0]
        vil_dec = lens_pop[:, 3] - (1-Dl_over_Di)*v_obs_lens[:, 1] - Dl_over_Di*v_star_lens[:, 1]
        beta_l = r_l/(lens_pop[:, 4]*kpc)

        lens_pop_ecl_lon, lens_pop_ecl_lat = fn_eq_to_ecl_array(lens_pop[:, 0], lens_pop[:, 1]) # convert into ecliptic coordinates for the parallax template

        ### To find the stars around each lens use a pixelation scale of size approx. max(beta_l)/10 and keep stars withing n_betat*max(beta_l)
        max_beta_l = max(np.max(beta_l), min_beta_t)
        n = round(math.log(np.sqrt(np.pi/3)/(0.1*max_beta_l), 2)); nside = 2**n; 
        q_pix = np.asarray(hp.ang2pix(nside, data_ra, data_dec, nest=True, lonlat=True)) 

        for i, l in enumerate(lens_pop):
            ### Find stars around the lens
            nb_lens_i = hp.query_disc(nside, vec_lens_array[i], n_betat*max_beta_l, inclusive=True, nest=True)
            stars_in = ((q_pix >= nb_lens_i[0]) & (q_pix <= nb_lens_i[-1])) # first reduce the total number of stars
            nb_stars = np.isin(q_pix[stars_in], nb_lens_i, assume_unique=False, invert=False) # keep only stars within the neighboring pixels  

            ### Proper motion template
            beta_it = angular_sep(lens_pop[i, 0]*degree, lens_pop[i, 1]*degree, data_ra[stars_in][nb_stars]*degree, data_dec[stars_in][nb_stars]*degree)
            mura_tilde, mudec_tilde, mu_sq = dipole_mf(beta_l[i], beta_it)
            mu_signal = (1-Dl_over_Di[i])*4*GN*M_l*vil_ra[i]/r_l**2*mura_tilde/(mas/Year) + (1-Dl_over_Di[i])*4*GN*M_l*vil_dec[i]/r_l**2*mudec_tilde/(mas/Year) # in mas/y

            data_pmra_sim[np.where(stars_in)[0][nb_stars]] += mu_signal[:, 0] # this way of accessing the elements of data_pmra_sim does not make a copy of the array
            data_pmdec_sim[np.where(stars_in)[0][nb_stars]] += mu_signal[:, 1]

            ### Parallax template
            beta_it_ecl = angular_sep(lens_pop_ecl_lon[i]*degree, lens_pop_ecl_lat[i]*degree, data_ecl_lon[stars_in][nb_stars]*degree, data_ecl_lat[stars_in][nb_stars]*degree)
            par_t = parallax_mf(beta_l[i], beta_it_ecl, np.sin(lens_pop_ecl_lat[i]*degree)) 

            data_parallax_sim[np.where(stars_in)[0][nb_stars]] += (1-Dl_over_Di[i])*4*GN*M_l*AU/r_l**2*par_t/mas # in mas
   
    ### Add signal to the data
    data['pmra_sim'] += data_pmra_sim; data['pmdec_sim'] += data_pmdec_sim; data['parallax_sim'] += data_parallax_sim; 
    
    return None

## Proper motion field subtraction - functions edited to work for the simulated quantities

In [15]:
def fn_prepare_back_sub(data, disc_center, disc_radius, beta_kernel_sub):
    """Prepare the data for the background motion subtraction"""
    ### Pixelation at approx 1/3 of beta_kernel
    n = round(math.log(np.sqrt(np.pi/3)/(beta_kernel_sub/3), 2))   
    nside = 2**n; npix = hp.nside2npix(nside);

    vec = hp.pix2vec(nside, hp.ang2pix(nside, disc_center[0], disc_center[1], nest=True, lonlat=True), nest=True)
    disc_pix = hp.query_disc(nside, vec, disc_radius, nest=True, inclusive=True) # pixels on the sky within the selected disc
    
    ### Stars healpy pixel number
    q_pix = np.asarray(hp.ang2pix(nside, data['ra'].to_numpy(), data['dec'].to_numpy(), nest=True, lonlat=True)) # healpy pixel number of the stars
    data.loc[:, ('q_pix_{}'.format(n))] = q_pix    
    
    ### Find neighboring pixels for each pixel
    nb_pixel_list = fn_nb_pixel(disc_pix, 3*beta_kernel_sub, nside, nest=True)

    return disc_pix, nb_pixel_list, n

In [16]:
def fn_back_field_sub(data, disc_pix, nb_pixel_array, n, beta_kernel=0.1*degree):
    """Creates a local map of the pm field and the parallax using a gaussian distance kenerl and subtracts the mean fields from each star pm and parallax"""
    
    nside = 2**n; npix = hp.nside2npix(nside);
    
    ### Pixelate stars using dataframe groupby
    old_pmra = data['pmra_sim'].to_numpy(); old_pmdec = data['pmdec_sim'].to_numpy(); old_parallax = data['parallax_sim'].to_numpy();
    data.drop(labels=['pmra_sim', 'pmdec_sim', 'parallax_sim'], axis="columns", inplace=True) 

    df_hist = pd.DataFrame({'q_pix_{}'.format(n):data['q_pix_{}'.format(n)].to_numpy(), 
                            'ra':data['ra'].to_numpy(), 'dec':data['dec'].to_numpy(),
                            'weighted_pmra':old_pmra/data['pmra_error'].to_numpy()**2, 
                            'weighted_pmdec':old_pmdec/data['pmdec_error'].to_numpy()**2, 
                            'weighted_parallax':old_parallax/data['parallax_error'].to_numpy()**2,
                            'pmra_w':1/data['pmra_error'].to_numpy()**2, 'pmdec_w':1/data['pmdec_error'].to_numpy()**2,
                            'parallax_w':1/data['parallax_error'].to_numpy()**2}).groupby(by=['q_pix_{}'.format(n)], as_index=False).sum()
            
    occ_pix = df_hist['q_pix_{}'.format(n)].to_numpy() # uniquely occupied pixels
    pix_count = np.bincount(data['q_pix_{}'.format(n)].to_numpy()); # number of stars per pixel  
    filled_pix_count = pix_count[pix_count>0]
    
    ### Full sky pixel arrays
    disc_pix_ra, disc_pix_dec = hp.pix2ang(nside, disc_pix, nest=True, lonlat=True) # coordinates of the selected pixels
    all_mean_coord = np.zeros((npix, 2)); 
    all_mean_coord[disc_pix] = np.array([disc_pix_ra, disc_pix_dec]).T # set pix coordinates to the pix center
    all_mean_coord[occ_pix] = np.array([df_hist['ra'].to_numpy()/filled_pix_count, df_hist['dec'].to_numpy()/filled_pix_count]).T # set pix coordinates to the mean coordinate

    all_mean_pm = np.zeros((npix, 2)); all_mean_parallax = np.zeros(npix) 
    all_mean_pm[occ_pix] = np.array([df_hist['weighted_pmra'].to_numpy()/df_hist['pmra_w'].to_numpy(), 
                                     df_hist['weighted_pmdec'].to_numpy()/df_hist['pmdec_w'].to_numpy()]).T    
    all_mean_parallax[occ_pix] = df_hist['weighted_parallax'].to_numpy()/df_hist['parallax_w'].to_numpy()
    
    n_disc_pix = len(disc_pix)
    pm_gauss = np.zeros((n_disc_pix, 2)); parallax_gauss = np.zeros(n_disc_pix)

    ### Loop over pixels
    for i in range(n_disc_pix):
        nb_pix = nb_pixel_array[i]     
        rel_distance_sq = angular_sep_magn_sq(all_mean_coord[disc_pix[i], 0]*degree, all_mean_coord[disc_pix[i], 1]*degree, 
                                              all_mean_coord[nb_pix, 0]*degree, all_mean_coord[nb_pix, 1]*degree)/(2*beta_kernel**2)
        gauss_weights = np.exp(-rel_distance_sq); sum_gauss_weights = sum(gauss_weights)
        pm_gauss[i, 0] = sum(all_mean_pm[nb_pix, 0]*gauss_weights)/sum_gauss_weights  # gaussian weighted mean pm in ra
        pm_gauss[i, 1] = sum(all_mean_pm[nb_pix, 1]*gauss_weights)/sum_gauss_weights  # gaussian weighted mean pm in dec
        parallax_gauss[i] = sum(all_mean_parallax[nb_pix]*gauss_weights)/sum_gauss_weights
        
    ### Interpolation of the velocity field
    pmra_interp = griddata((all_mean_coord[disc_pix, 0], all_mean_coord[disc_pix, 1]), pm_gauss[:, 0], (data['ra'].to_numpy(), data['dec'].to_numpy()), method='linear', fill_value=0)
    pmdec_interp = griddata((all_mean_coord[disc_pix, 0], all_mean_coord[disc_pix, 1]), pm_gauss[:, 1], (data['ra'].to_numpy(), data['dec'].to_numpy()), method='linear', fill_value=0)
    parallax_interp = griddata((all_mean_coord[disc_pix, 0], all_mean_coord[disc_pix, 1]), parallax_gauss, (data['ra'].to_numpy(), data['dec'].to_numpy()), method='linear', fill_value=0)
        
    data.insert(len(data.columns), 'pmra_sim', old_pmra - pmra_interp); data.insert(len(data.columns), 'pmdec_sim', old_pmdec - pmdec_interp)
    data.insert(len(data.columns), 'parallax_sim', old_parallax - parallax_interp); 
    
    return None

In [17]:
def fn_rem_outliers(data, pm_esc, D_s, n_sigma_out=3):
    """Remove stars with pm or parallax more than n_sigma_out sigma away from the expected value"""
    """Returns cleaned stars and fraction of outliers removed"""
    old_len = len(data)
    new_data = data[( np.sqrt(data['pmra_sim'].to_numpy()**2 + data['pmdec_sim'].to_numpy()**2) < 
                      (pm_esc + n_sigma_out*np.sqrt(data['pmra_error'].to_numpy()**2 + data['pmdec_error'].to_numpy()**2)) ) &
                    ( np.abs(data['parallax_sim'].to_numpy()) < (1/D_s + n_sigma_out*data['parallax_error'].to_numpy()) )]
    return new_data, 1-len(new_data)/old_len

In [18]:
def fn_rem_edges(data, disc_center, disc_radius):
    """Keep only stars within disc_radius of the disc_center, to remove the edges"""
    center_sky_coord = SkyCoord(ra = disc_center[0] * u.deg, dec = disc_center[1] * u.deg)
    data_sky_coord = SkyCoord(ra = data['ra'].to_numpy() * u.deg, dec = data['dec'].to_numpy() * u.deg)
    data_r = data_sky_coord.separation(center_sky_coord).value*degree
     
    return data[data_r < disc_radius]    

## Effective weights - function edited to work for the simulated quantities

In [19]:
def fn_effective_w(data, disc_center, gmag_bin_size=0.1, rad_bin_size=1):
    """Compute effective pm and parallax dispersion in gmag and radial bins"""
    
    ### Bin in g magnitude and radial distance from the center
    data_g = data['phot_g_mean_mag'].to_numpy()
    min_g, max_g = np.min(data_g), np.max(data_g)
    #bins_g = np.arange(min_g, max_g+gmag_bin_size, gmag_bin_size)
    ### Using g magnitude bins with approximately equal number of stars per bin, to avoid bins with a low number of stars    
    n_bins_g = int(np.ceil((max_g-min_g)/gmag_bin_size))
    bins_g = np.interp(np.linspace(0, len(data_g+1), n_bins_g + 1), np.arange(len(data_g)+1), np.append(np.sort(data_g), max_g*1.001))  # make sure that the last bin includes max_g
    q_bin_g = np.digitize(data_g, bins_g)-1         
    
    center_sky_coord = SkyCoord(ra = disc_center[0] * u.deg, dec = disc_center[1] * u.deg)
    data_sky_coord = SkyCoord(ra = data['ra'].to_numpy() * u.deg, dec = data['dec'].to_numpy() * u.deg)
    data_r = data_sky_coord.separation(center_sky_coord).value
    bins_r = np.arange(0, np.max(data_r)+rad_bin_size, rad_bin_size)
    q_bin_r = np.digitize(data_r, bins_r)-1     
    
    ### Histograms with mean pm and parallax dispersion per bin
    counts = np.histogram2d(data_g, data_r, bins=[bins_g, bins_r], weights=None)[0]
    pm_sq = np.histogram2d(data_g, data_r, bins=[bins_g, bins_r], weights=(data['pmra_sim'].to_numpy()**2 + data['pmdec_sim'].to_numpy()**2))[0]
    par_sq = np.histogram2d(data_g, data_r, bins=[bins_g, bins_r], weights=(data['parallax_sim'].to_numpy()**2))[0]

    sigma_pm_eff_hist = np.sqrt(np.divide(pm_sq, counts, out=np.zeros_like(pm_sq), where=counts!=0))
    sigma_par_eff_hist = np.sqrt(np.divide(par_sq, counts, out=np.zeros_like(par_sq), where=counts!=0))

    ### Set to zero for bins that have less than 30 stars
    sigma_pm_eff_hist[counts < 30] = 0
    sigma_par_eff_hist[counts < 30] = 0
    
    ### Add effective error columns (for each star, take the max between the instrumental and effective dispersion)
    data.insert(len(data.columns), 'pm_eff_error', np.max(np.array([sigma_pm_eff_hist[q_bin_g, q_bin_r], np.sqrt(data['pmra_error'].to_numpy()**2 + data['pmdec_error'].to_numpy()**2)]), axis=0))
    data.insert(len(data.columns), 'parallax_eff_error', np.max(np.array([sigma_par_eff_hist[q_bin_g, q_bin_r], data['parallax_error'].to_numpy()]), axis=0))
    
    return None

## Template scan

In [20]:
def fn_template_scan(nside, scan_pix, q_pix):
    """Compute the template at the locations given by coarse_scan_pix"""
    """Includes pm and parallax templates"""
        
    scan_coord = hp.pix2ang(nside, scan_pix, nest=True, lonlat=True) # coordinates of the template locations
    scan_coord_ecl = fn_eq_to_ecl_array(scan_coord[0], scan_coord[1]) # convert into ecliptic coordinates for the parallax template

    ### Cartesian vectors for each pixel in scan_pix
    vec_pix_x, vec_pix_y, vec_pix_z = hp.pix2vec(nside, scan_pix, nest=True) 
    vec_array = np.array([vec_pix_x, vec_pix_y, vec_pix_z]).T
    
    n_loc = len(scan_pix)
    tau_mu_ra, tau_mu_dec, tau_mu_norm = np.zeros(n_loc), np.zeros(n_loc), np.zeros(n_loc)
    tau_par, tau_par_norm = np.zeros(n_loc), np.zeros(n_loc)
    
    for i in range(n_loc):
        nb_pix_i = hp.query_disc(nside, vec_array[i], n_betat*beta_t, inclusive=True, nest=True) # disc around the template position 

        stars_in = ((q_pix >= nb_pix_i[0]) & (q_pix <= nb_pix_i[-1])) # first reduce the total number of stars   
        nb_stars = np.isin(q_pix[stars_in], nb_pix_i, assume_unique=False, invert=False) # keep only stars within the neighboring pixels 
        #nb_stars = np.intersect1d(q_pix[stars_in], nb_pix_i, assume_unique=False, return_indices=True)[1] # this does not work because it returns only the unique list of common values! we also want repeated entries

        ### Pm template
        beta_it = angular_sep(scan_coord[0][i]*degree, scan_coord[1][i]*degree, data_ra[stars_in][nb_stars]*degree, data_dec[stars_in][nb_stars]*degree)
        mu_ra, mu_dec, mu_sq = dipole_mf(beta_t, beta_it)

        tau_mu_norm[i] = np.sqrt(sum(mu_sq/pm_w_sq[stars_in][nb_stars])) ## normalization
        tau_mu_ra[i] = sum((mu_ra[:, 0]*weighted_pmra[stars_in][nb_stars] + mu_ra[:, 1]*weighted_pmdec[stars_in][nb_stars]))
        tau_mu_dec[i] = sum((mu_dec[:, 0]*weighted_pmra[stars_in][nb_stars] + mu_dec[:, 1]*weighted_pmdec[stars_in][nb_stars]))
                
        ### Parallax template
        beta_it_ecl = angular_sep(scan_coord_ecl[0][i]*degree, scan_coord_ecl[1][i]*degree, data_ecl_lon[stars_in][nb_stars]*degree, data_ecl_lat[stars_in][nb_stars]*degree)
        par_t = parallax_mf(beta_t, beta_it_ecl, np.sin(scan_coord_ecl[1][i]*degree)) 
        
        tau_par_norm[i] = np.sqrt(sum(par_t**2/par_w_sq[stars_in][nb_stars])) ## normalization
        tau_par[i] =  sum(par_t*weighted_par[stars_in][nb_stars])
            
    return np.array([scan_coord[0], scan_coord[1], tau_mu_ra, tau_mu_dec, tau_mu_norm, tau_par, tau_par_norm]).T

## Test statistic

In [21]:
def fn_chi_sq(M_l, r_l, beta_t, tau_values):
    """Returns the chi_sq (optimal global test statistic) for each of the tau_values. The minimum should be retained."""
    """Returns the location, the beta_t value and the chi_sq including both the proper motion and parallax templates and the proper motion only"""
    [ind_ra, ind_dec, ind_tau_ra, ind_tau_dec, ind_n, ind_tau_par, ind_par_n] = range(7)
    ### Converting to natural units
    n_mu_list = tau_values[:, ind_n]*(Year/mas) 
    t_mu_ra_list, t_mu_dec_list = tau_values[:, ind_tau_ra]*(Year/mas), tau_values[:, ind_tau_dec]*(Year/mas)
    t_mu_sq = t_mu_ra_list**2 + t_mu_dec_list**2
    n_p_list = tau_values[:, ind_par_n]/mas; t_p_list = tau_values[:, ind_tau_par]/mas
    v0 = -fn_obs_vel(tau_values[:, ind_ra]*degree, tau_values[:, ind_dec]*degree) # prefered direction for the velocity template
    v0_sq = v0[:, 0]**2 + v0[:, 1]**2
    tau_dot_v0 = t_mu_ra_list*v0[:, 0] + t_mu_dec_list*v0[:, 1] 

    tau_skycoord = SkyCoord(tau_values[:, ind_ra]*u.deg, tau_values[:, ind_dec]*u.deg)
    rho_beta_t_contr = -2*np.log(r_l**3/M_l*fn_rho_dm_array(np.full(len(tau_values), r_l)/beta_t, tau_skycoord.galactic.l.rad, tau_skycoord.galactic.b.rad)/beta_t**4)
    
    C_l = 4*GN*M_l/r_l**2
    Cl_sigmav_Nmu = C_l*sigma_vl*n_mu_list
    t_mu_sq_over_n_sq = np.divide(t_mu_sq, n_mu_list**2, out=np.zeros_like(n_mu_list), where=n_mu_list!=0)
    tau_dot_over_n = np.divide(tau_dot_v0, n_mu_list*sigma_vl, out=np.zeros_like(n_mu_list), where=n_mu_list!=0)
    mu_templ_contr = -(Cl_sigmav_Nmu)/(1+Cl_sigmav_Nmu**2)*( Cl_sigmav_Nmu*(t_mu_sq_over_n_sq - v0_sq/sigma_vl**2) + 2*tau_dot_over_n )
           
    Cl_Au_Np = C_l*AU*n_p_list
    t_p_over_n = np.divide(t_p_list, n_p_list, out=np.zeros_like(n_p_list), where=n_p_list!=0)
    p_templ_contr = -2*Cl_Au_Np*( t_p_over_n - Cl_Au_Np/2 )
    
    return np.array([tau_values[:, ind_ra], tau_values[:, ind_dec], np.full((len(tau_values)), beta_t), rho_beta_t_contr + mu_templ_contr + p_templ_contr, rho_beta_t_contr + mu_templ_contr]).T

## Simulation template analysis

In [22]:
def fn_run_analysis(beta_t, M_l, r_l, n_lens, lens_pop):
    """Computes the templates for the given value of beta_t and the given list of lenses. Returns the list of chi^2 for each location."""
    ### Coarse pixelation scale
    n_coarse = math.ceil(math.log(np.sqrt(np.pi/3)/beta_t, 2)); npix_coarse = hp.nside2npix(2**n_coarse); 
    beta_pix_coarse = np.sqrt(4*np.pi / npix_coarse)    
    
    ### Take 5 locations around the lens (ra, dec) where to compute the template using the step of the fine pixelation
    ### Taking 20 locations per lens
    beta_steps = np.array([-2, -1, 0, 1, 2])*beta_step*beta_pix_coarse/degree
    template_scan_ra, template_scan_dec = [], []

    if n_lens==1:
        x_list = lens_pop[0] + beta_steps/np.cos(lens_pop[1]*degree)
        y_list = lens_pop[1] + beta_steps

        xy_grid = np.meshgrid(x_list, y_list, indexing='xy')
        x_grid_flat = xy_grid[0].flatten(); y_grid_flat = xy_grid[1].flatten()

        xy_dist = angular_sep_scalar(lens_pop[0]*degree, lens_pop[1]*degree, x_grid_flat*degree, y_grid_flat*degree)
        template_scan_ra.extend(x_grid_flat[xy_dist < beta_pix_coarse]); template_scan_dec.extend(y_grid_flat[xy_dist < beta_pix_coarse])
        
    else:
        for i in range(len(lens_pop)):
            x_list = lens_pop[i, 0] + beta_steps/np.cos(lens_pop[i, 1]*degree)
            y_list = lens_pop[i, 1] + beta_steps

            xy_grid = np.meshgrid(x_list, y_list, indexing='xy')
            x_grid_flat = xy_grid[0].flatten(); y_grid_flat = xy_grid[1].flatten()

            xy_dist = angular_sep_scalar(lens_pop[i, 0]*degree, lens_pop[i, 1]*degree, x_grid_flat*degree, y_grid_flat*degree)
            template_scan_ra.extend(x_grid_flat[xy_dist < beta_pix_coarse]); template_scan_dec.extend(y_grid_flat[xy_dist < beta_pix_coarse])
        
    ### Fine pixelation of size approx. beta_t/10 (can be a bit larger than beta_t/10, so using round is fine)
    n = round(math.log(np.sqrt(np.pi/3)/(0.1*beta_t), 2)); nside = 2**n; 
    template_scan_pix = np.unique(hp.ang2pix(nside, template_scan_ra, template_scan_dec, nest=True, lonlat=True)) 
    n_locations = len(template_scan_pix)

    print('\nTemplate scan for beta_t = '+str(beta_t/degree)+' deg.')
    print('Number of template locations: '+str(n_locations))
    sys.stdout.flush()
    
    q_pix = np.asarray(hp.ang2pix(nside, data_ra, data_dec, nest=True, lonlat=True)) # healpy pixel number of the stars, needed to find stars near a specific template location
    
    ### Compute the template for "step" number of location
    tau_values = fn_template_scan(nside, template_scan_pix, q_pix)
    
    print('Template scan completed. Computing the chi_sq...')
    sys.stdout.flush()
    chi_sq = fn_chi_sq(M_l, r_l, beta_t, tau_values)
    
    return chi_sq

# Run

## Preamble

In [23]:
tic = tictoc()

### Define the sky patches used for the analysis
### Parameters taken from Gaia Early Data Release 3: Structure and properties of the Magellanic Clouds (see Table 4)
LMC_sky_patch = sky_patch(81.28, -69.78, 5*degree, 50*kpc, 'LMC_disc_5', np.array([1.871, 0.391]))
SMC_sky_patch = sky_patch(12.80, -73.15, 4*degree, 60*kpc, 'SMC_disc_4', np.array([0.686, -1.237]))

all_sky_patches = [LMC_sky_patch, SMC_sky_patch]

In [27]:
### Read in paramters from the command line
### Example of how to run the python script with the simulation n.0 for the paramter space point (M_l, r_l, f_l) = (10^8 M_solar, 1 pc, 1)
### python dm_simulation_script LMC 80 30 30 0
sky_patch_name = sys.argv[1] #'LMC'
M_l = math.pow(10, float(sys.argv[2])/10)*MSolar
r_l = math.pow(10, (-3 + float(sys.argv[3])/10))*pc
f_l = math.pow(10, (-3 + float(sys.argv[4])/10))

In [25]:
### Define sky patch to use in this simulation
if sky_patch_name == 'LMC':    
    sim_sky_patch = LMC_sky_patch
    print(' ********** Running the '+sys.argv[5]+'th simulation analysis on the LMC **********\n')
elif sky_patch_name == 'SMC':
    sim_sky_patch = SMC_sky_patch
    print(' ********** Running the '+sys.argv[5]+'th simulation analysis on the SMC **********\n')
else:
    print('ERROR: wrong name provided for the sky patch!')
sys.stdout.flush()

In [27]:
disc_radius = sim_sky_patch.disc_radius
disc_center = np.array([sim_sky_patch.center_ra, sim_sky_patch.center_dec])
data_file_name = sim_sky_patch.data_file_name
result_file_name = data_file_name+'_'+sys.argv[2]+'_'+sys.argv[3]+'_'+sys.argv[4]+'_'+sys.argv[5]
#result_file_name = data_file_name+'_80_30_32_1'
print('Result will be saved in the file '+result_file_name)

Result will be saved in the file LMC_disc_5_80_30_32_1


In [28]:
### Parameters for data cleaning
beta_kernel_sub_0 = 0.1*degree; # gaussian kernel for background subtraction 
beta_kernel_sub = 0.06*degree; # 0.1*degree; gaussian kernel for background subtraction 
pm_esc = 0.2; D_s = 50; n_sigma_out_0 = 5; n_sigma_out = 3; # escape velocity, distance of the stars given in kpc, number of sigmas for outlier removal
n_iter_sub = 3; # number of iterations for the background subtraction
disc_radius_no_edge = disc_radius - beta_kernel_sub_0 - (n_iter_sub+1)*beta_kernel_sub
gmag_bin_size=0.1; rad_bin_size=1 # for the effective weights

n_betat=3.5;
min_beta_t = 0.002*degree # for point-like lenses, inject signal on stars around the lens within n_betat*min_beta_t 
beta_step = 1/9; # beta_t step used in the fine pixelation, needed to determine positions around the lens locations where to compute the template

n_lens_max = 200 # maximum number of the closest lenses to keep for the signal injection and analysis

In [29]:
### Load list of beta_t values and convert to radians
beta_t_list = np.load(ListDir+'beta_t_list.npy')/10000*degree 
### Optimal value of beta_t for the lens population parameters
beta_t_opt = fn_beta_t_opt(M_l, r_l, f_l, all_sky_patches)

### Find the beta_t values from the beta_t_list closest to the optimal beta_t
beta_t_opt_ind = np.argmin(np.abs(np.array(beta_t_list) - beta_t_opt))
if beta_t_opt_ind==0:
    beta_t_opt_list = beta_t_list[:beta_t_opt_ind+2]
elif beta_t_opt_ind==(len(beta_t_list)-1):
    beta_t_opt_list = beta_t_list[beta_t_opt_ind-1:]
else:
    beta_t_opt_list = beta_t_list[beta_t_opt_ind-1:beta_t_opt_ind+2]

print('\nOptimal beta_t value:', str(beta_t_opt/degree)[:7], ' deg.') 
print('Computing the template for beta_t =', str(str(beta_t_opt_list/degree)))
sys.stdout.flush()


Optimal beta_t value: 0.00473  deg.
Computing the template for beta_t = [0.005  0.0045 0.004 ]


In [30]:
### Loading the data -- loading an npy file is much faster than loading the csv file with pd.rad_csv
data_np = np.load(DataDir+data_file_name+'_clean.npy')
columns_df = ['ra', 'dec', 'pmra', 'pmdec', 'parallax', 'pmra_error', 'pmdec_error', 'parallax_error', 'phot_g_mean_mag', 'ecl_lon', 'ecl_lat', 'pmra_sub', 'pmdec_sub', 'parallax_sub']
data = pd.DataFrame(data_np, columns=columns_df)
data.shape

(14730230, 14)

## Execution

In [31]:
### Generating the lens population
n_lens, lens_pop = fn_lens_population(M_l, f_l, sim_sky_patch)
print(n_lens, 'lenses in front of the stellar target.')
sys.stdout.flush()

### Sort lenses based on their distance and keep only the n_lens_max closest ones
if n_lens > n_lens_max:
    print(len(lens_pop), 'lenses. Retaining only the closest', n_lens_max)
    sys.stdout.flush()
    lens_ind_sort = np.argsort(lens_pop[:, -1])
    lens_pop = lens_pop[lens_ind_sort[:n_lens_max]]

14 lenses in front of the stellar target.


In [32]:
columns_res = ['ra', 'dec', 'beta_t', 'min_chi_sq', 'min_chi_sq_mu_only']

if n_lens == 0: ### If there are no lenses in front of the stellar target, skip the analysis and set the resulting chi^2 to zero
    res_df = pd.DataFrame(np.zeros((2, 5)), columns=columns_res)
    res_df.to_csv(ListResDir+result_file_name+'.csv', index=False)
    toc = tictoc()    
    print('Simulation done in', str(toc - tic), 's.')        
    sys.stdout.flush()          
else:
    ### Injecting the noise
    fn_noise_inj(data, disc_center, gmag_bin_size, rad_bin_size, noise=True)
    print('Injecting the signal..')
    sys.stdout.flush()
    fn_signal_inj(data, M_l, r_l, n_lens, lens_pop, sim_sky_patch, n_betat, min_beta_t)
    
    ### Prepare the mock data for the iterative background subtraction and outlier removal
    disc_pix, nb_pixel_list, n = fn_prepare_back_sub(data, disc_center, disc_radius, beta_kernel_sub)

    print('\nBackground subtraction..')
    sys.stdout.flush()    
    ### Iterative background subtraction and outlier removal
    for i in range(n_iter_sub):
        fn_back_field_sub(data, disc_pix, nb_pixel_list, n, beta_kernel=beta_kernel_sub) ### sub=True can be used only after this function has been already called once with sub=False
        data, f_out = fn_rem_outliers(data, pm_esc, D_s, n_sigma_out)
        print('Iter '+str(i)+' -- fraction of outliers removed: '+str(f_out*100)[:7]+' %')

    fn_back_field_sub(data, disc_pix, nb_pixel_list, n, beta_kernel=beta_kernel_sub)

    ### Remove stars at the boundary to avoid edge effect due to gaussian kernel field subtraction
    data = fn_rem_edges(data, disc_center, disc_radius_no_edge)
    
    ### Compute the effective weights
    fn_effective_w(data, disc_center, gmag_bin_size, rad_bin_size)

    ### Quantities used to compute the template
    data_ra, data_dec = data['ra'].to_numpy(), data['dec'].to_numpy()
    data_ecl_lon, data_ecl_lat = data['ecl_lon'].to_numpy(), data['ecl_lat'].to_numpy()
    pm_w_sq = (data['pm_eff_error'].to_numpy())**2
    weighted_pmra = data['pmra_sim'].to_numpy()/data['pm_eff_error'].to_numpy()**2
    weighted_pmdec = data['pmdec_sim'].to_numpy()/data['pm_eff_error'].to_numpy()**2
    par_w_sq = (data['parallax_eff_error'].to_numpy())**2
    weighted_par = data['parallax_sim'].to_numpy()/data['parallax_eff_error'].to_numpy()**2
        
    ### Run the analysis on the mock data to compute the template at the lens locations and compute the chi^2
    chi_sq = [];
    for beta_t in beta_t_opt_list:
        chi_sq.extend(fn_run_analysis(beta_t, M_l, r_l, n_lens, lens_pop)); 
    chi_sq = np.array(chi_sq)
    
    ### Save the result into a file
    res_df = pd.DataFrame(np.array([chi_sq[np.argmin(chi_sq[:, 3])], chi_sq[np.argmin(chi_sq[:, 4])]]), columns=columns_res)
    res_df.to_csv(ListResDir+result_file_name+'.csv', index=False)

    toc = tictoc()    
    print('Simulation done in', str(toc - tic), 's.')        
    sys.stdout.flush()

Injecting the data-driven noise..
Injecting the signal..

Background subtraction..
Iter 0 -- fraction of outliers removed: 9.23113 %
Iter 1 -- fraction of outliers removed: 0.02300 %
Iter 2 -- fraction of outliers removed: 0.00914 %

Template scan for beta_t = 0.005 deg.
Number of template locations: 294
Template scan completed. Computing the chi_sq...

Template scan for beta_t = 0.0045 deg.
Number of template locations: 294
Template scan completed. Computing the chi_sq...

Template scan for beta_t = 0.004 deg.
Number of template locations: 294
Template scan completed. Computing the chi_sq...
Simulation done in 914.2507839202881 s.
